In [1]:
%tensorflow_version 1.x
import tensorflow.contrib.tensorrt as trt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as Net
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np
import os

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
mobilenet_v2 = tf.keras.applications.MobileNetV2(weights='imagenet')
tf.saved_model.save(mobilenet_v2, '/content/model_dirs')

INFO:tensorflow:Assets written to: /content/model_dirs/assets


In [20]:
img = tf.keras.preprocessing.image.load_img('/content/elephant.png', target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

In [21]:
preds = mobilenet_v2.predict(x)
print('Predicted:', tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=3)[0])

Predicted: [('n02504013', 'Indian_elephant', 0.61320823), ('n01871265', 'tusker', 0.29030368), ('n02504458', 'African_elephant', 0.016726842)]


In [22]:
import time
def time_my_model(model, data):
    times = []
    for i in range(20):
        start_time = time.time()
        one_prediction = model.predict(data)
        delta = (time.time() - start_time)
        times.append(delta)
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    print('average(sec):{:.2f},fps:{:.2f}'.format(mean_delta, fps))

In [23]:
for i in range(5):
  time_my_model(mobilenet_v2, x)

average(sec):0.07,fps:14.41
average(sec):0.07,fps:14.63
average(sec):0.07,fps:14.84
average(sec):0.07,fps:14.65
average(sec):0.07,fps:14.66


In [24]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(precision_mode='FP16')
converter = trt.TrtGraphConverterV2(
        input_saved_model_dir='/content/model_dirs/',
        conversion_params=conversion_params,
    )
converter.convert()
converter.save(output_saved_model_dir='/content/trt_m2_dirs')

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: /content/trt_m2_dirs/assets


In [26]:
root = tf.saved_model.load('/content/trt_m2_dirs')
concrete_func = root.signatures['serving_default']
import numpy as np
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
concrete_func.structured_outputs

{'predictions': TensorSpec(shape=(None, 1000), dtype=tf.float32, name='predictions')}

In [29]:
for i in range(5):
  labeling = concrete_func(tf.constant(x.astype('float32')))
  activations = tf.nn.softmax(labeling['predictions'])
  print(imagenet_labels[np.argsort(activations)[0,::-1][:5]+1])


['Indian elephant' 'tusker' 'African elephant' 'triceratops' 'web site']
['Indian elephant' 'tusker' 'African elephant' 'triceratops' 'web site']
['Indian elephant' 'tusker' 'African elephant' 'triceratops' 'web site']
['Indian elephant' 'tusker' 'African elephant' 'triceratops' 'web site']
['Indian elephant' 'tusker' 'African elephant' 'triceratops' 'web site']


In [30]:
def time_trt_model():
    image_input = tf.constant(x.astype('float32'))
    times = []
    for i in range(20):
        start_time = time.time()
        one_prediction = concrete_func(input_1=image_input)
        delta = (time.time() - start_time)
        times.append(delta)
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    print('average(sec):{:.2f},fps:{:.2f}'.format(mean_delta, fps))

In [33]:
img = tf.keras.preprocessing.image.load_img('/content/elephant.png', target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

In [34]:
for i in range(5):
  time_trt_model()

average(sec):0.04,fps:25.30
average(sec):0.04,fps:24.98
average(sec):0.04,fps:25.51
average(sec):0.04,fps:24.96
average(sec):0.04,fps:25.03


In [36]:
!du --all -h '/content/model_dirs'

3.9M	/content/model_dirs/saved_model.pb
108K	/content/model_dirs/variables/variables.data-00001-of-00002
14M	/content/model_dirs/variables/variables.data-00000-of-00001
98M	/content/model_dirs/variables/variables.data-00000-of-00002
20K	/content/model_dirs/variables/variables.index
112M	/content/model_dirs/variables
4.0K	/content/model_dirs/assets
99M	/content/model_dirs/model.pb
99M	/content/model_dirs/model
312M	/content/model_dirs


In [37]:
!du --all -h '/content/trt_m2_dirs'

31M	/content/trt_m2_dirs/saved_model.pb
14M	/content/trt_m2_dirs/variables/variables.data-00000-of-00001
20K	/content/trt_m2_dirs/variables/variables.index
14M	/content/trt_m2_dirs/variables
4.0K	/content/trt_m2_dirs/assets
45M	/content/trt_m2_dirs
